In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image dimensions and batch size
img_size = 48
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    r"C:\Spam\archive\train",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    r"C:\Spam\archive\train",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

# Model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(train_generator, validation_data=val_generator, epochs=20)

# Save model
model.save('emotion_detection_model.h5')


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


C:\Users\somuk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\somuk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 346s 478ms/step - accuracy: 0.2456 - loss: 1.8220 - val_accuracy: 0.3675 - val_loss: 1.6311
Epoch 2/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 97s 135ms/step - accuracy: 0.3798 - loss: 1.5851 - val_accuracy: 0.4480 - val_loss: 1.4649
Epoch 3/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - accuracy: 0.4425 - loss: 1.4391 - val_accuracy: 0.4785 - val_loss: 1.3614
Epoch 4/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 18s 26ms/step - accuracy: 0.4801 - loss: 1.3548 - val_accuracy: 0.4970 - val_loss: 1.3107
Epoch 5/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - accuracy: 0.4958 - loss: 1.3030 - val_accuracy: 0.5206 - val_loss: 1.2599
Epoch 6/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - accuracy: 0.5299 - loss: 1.2351 - val_accuracy: 0.5358 - val_loss: 1.2302
Epoch 7/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 23s 32ms/step - accuracy: 0.5428 - loss: 1.2129 - val_accuracy: 0.5233 - val_loss: 1.2500
Epoch 8/20
718/718 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.5569 - loss: 1.1642

In [14]:
import tensorflow as tf
import numpy as np
import cv2

# Load the saved model
model = tf.keras.models.load_model("emotion_detection_model.h5")
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    img = cv2.resize(img, (48, 48))  # Resize to match model input
    img = img / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img

# Define emotion labels (ensure it matches your training classes)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Load and preprocess the image
img_path = r"C:\Spam\archive\train\disgust\Training_69367360.jpg" # Replace with actual image path
processed_img = preprocess_image(img_path)

# Predict emotion
prediction = model.predict(processed_img)
predicted_class = np.argmax(prediction)  # Get class with highest probability
print("Predicted Emotion:", emotion_labels[predicted_class])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Predicted Emotion: Angry


In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

# Set image size and batch size
img_size = 48
batch_size = 32

# Data Augmentation and Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load train and validation datasets
train_generator = datagen.flow_from_directory(
    r"C:\Spam\archive\train",  
    target_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode='rgb',  # Convert grayscale to RGB
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    r"C:\Spam\archive\train",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode='rgb',  # Convert grayscale to RGB
    class_mode='categorical',
    subset='validation'
)

# Load Pretrained VGG16 Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze the VGG16 layers (keep learned features)
for layer in base_model.layers:
    layer.trainable = False

# Create new model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(train_generator, validation_data=val_generator, epochs=5)

# Save Model
model.save("improved_emotion_model.h5")
print("Model saved successfully!")


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Epoch 1/5


C:\Users\somuk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


718/718 ━━━━━━━━━━━━━━━━━━━━ 199s 274ms/step - accuracy: 0.2646 - loss: 1.8187 - val_accuracy: 0.3235 - val_loss: 1.6910
Epoch 2/5
718/718 ━━━━━━━━━━━━━━━━━━━━ 306s 426ms/step - accuracy: 0.3150 - loss: 1.7102 - val_accuracy: 0.3304 - val_loss: 1.6747
Epoch 3/5
718/718 ━━━━━━━━━━━━━━━━━━━━ 274s 381ms/step - accuracy: 0.3307 - loss: 1.6823 - val_accuracy: 0.3379 - val_loss: 1.6650
Epoch 4/5
718/718 ━━━━━━━━━━━━━━━━━━━━ 207s 288ms/step - accuracy: 0.3304 - loss: 1.6768 - val_accuracy: 0.3409 - val_loss: 1.6552
Epoch 5/5
718/718 ━━━━━━━━━━━━━━━━━━━━ 245s 342ms/step - accuracy: 0.3363 - loss: 1.6679 - val_accuracy: 0.3414 - val_loss: 1.6533


Model saved successfully!


In [9]:
import tensorflow as tf
import numpy as np
import cv2

# Load Trained Model
model = tf.keras.models.load_model("improved_emotion_model.h5")

# Define emotion labels (ensure it matches training classes)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Function to preprocess new images
def preprocess_image(img_path):
    img = cv2.imread(img_path)  # Read image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (48, 48))  # Resize
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Test Image Path (Change this)
img_path = r"C:\Spam\archive\test\angry\PrivateTest_2775163.jpg"

# Predict Emotion
processed_img = preprocess_image(img_path)
prediction = model.predict(processed_img)
predicted_class = np.argmax(prediction)

print("Predicted Emotion:", emotion_labels[predicted_class])
print("Prediction Probabilities:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Predicted Emotion: Happy
Prediction Probabilities: [[0.17458038 0.01921446 0.11295405 0.3916243  0.1125332  0.11869187
  0.07040177]]
